In [1]:
import torch
import torch.nn as nn

In [ ]:
#Creating a transformer block with all components
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.norm1 = LayerNorm()
        self.att = MultiHeadAttention()
        self.dropout = nn.Dropout()
        self.norm2 = LayerNorm()
        self.feedforward = FeedForward()

    def forward(self, x):
        